In [ ]:
#P1 
import pandas as pd
import subprocess

def run_bedtools_getfasta(bed_file, genome_fasta, output_file):
    """Uses bedtools getfasta to extract nucleotides based on the BED intervals."""
    command = f"bedtools getfasta -fi {genome_fasta} -bed {bed_file} -fo {output_file}"
    subprocess.run(command, shell=True, check=True)

def load_bed_files(file_paths):
    """Loads BED files from a list of file paths."""
    bed_frames = [pd.read_csv(file_path, sep='\t', header=None) for file_path in file_paths]
    return bed_frames

def convert_bed_to_nucleotides(bed_files, genome_fasta, output_file):
    """Converts BED files to nucleotide sequences using bedtools."""
    combined_bed = pd.concat(bed_files)
    combined_bed.to_csv("combined_bed_file.bed", sep="\t", header=False, index=False)
    run_bedtools_getfasta("combined_bed_file.bed", genome_fasta, output_file)

def filter_sequences(input_file, output_file, fixed_length):
    """Filters and crops nucleotide sequences to a fixed length.
    Discards sequences that are shorter or longer than the fixed length.
    Each sequence is written on a separate line.
    """
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        sequence = ""
        for line in infile:
            if line.startswith(">"):
                if len(sequence) == fixed_length:
                    # Write the trimmed sequence to the output file
                    outfile.write(sequence + "\n")
                # Reset sequence for the next record
                sequence = ""
            else:
                sequence += line.strip()
        # Write the last sequence after EOF
        if len(sequence) == fixed_length:
            outfile.write(sequence + "\n")

def create_negative_bed_file(bed_files, fixed_length, output_file):
    """Creates a negative BED file with regions not covered by the original BED files."""
    combined_bed = pd.concat(bed_files)
    combined_bed.sort_values(by=[0, 1], inplace=True)
    negative_bed = []
    
    # Iterate through consecutive rows to find the gaps
    for i in range(1, len(combined_bed)):
        prev_row = combined_bed.iloc[i - 1]
        curr_row = combined_bed.iloc[i]
        
        if prev_row[0] == curr_row[0]:  # Same chromosome
            gap_start = prev_row[2] + 1
            gap_end = curr_row[1] - 1
            
            # Create multiple fixed-length intervals from the gap
            while (gap_end - gap_start + 1) >= fixed_length:
                negative_bed.append([prev_row[0], gap_start, gap_start + fixed_length])  # Fixed here
                gap_start += fixed_length
    
    negative_bed_df = pd.DataFrame(negative_bed)
    negative_bed_df.to_csv(output_file, sep='\t', header=False, index=False)

def generate_negative_sequences(negative_bed_file, genome_fasta, output_file):
    """Generates nucleotide sequences for the negative regions."""
    run_bedtools_getfasta(negative_bed_file, genome_fasta, output_file)

def process_dnase_seq_data(bed_files_paths, genome_fasta, output_prefix, fixed_length):
    """Main function to process DNase-seq data."""
    # Step 1: Load BED files
    bed_files = load_bed_files(bed_files_paths)
    
    # Step 2: Convert to nucleotide sequences
    nucleotide_output_file = f"{output_prefix}_positive_atgc.txt"
    convert_bed_to_nucleotides(bed_files, genome_fasta, nucleotide_output_file)
    
    # Step 3: Filter sequences to a fixed length
    positive_fixed_output = f"{output_prefix}_positive_fixed_length.txt"
    filter_sequences(nucleotide_output_file, positive_fixed_output, fixed_length)
    
    # Step 4: Create negative BED file
    negative_bed_file = f"{output_prefix}_negative.bed"
    create_negative_bed_file(bed_files, fixed_length, negative_bed_file)
    
    # Step 5: Generate negative sequences
    negative_output_file = f"{output_prefix}_negative_atgc.txt"
    generate_negative_sequences(negative_bed_file, genome_fasta, negative_output_file)
    
    # Step 6: Filter negative sequences to the same length as positive sequences
    negative_fixed_output = f"{output_prefix}_negative_fixed_length.txt"
    filter_sequences(negative_output_file, negative_fixed_output, fixed_length)
    
    # Step 7: Trim headers from positive and negative fixed-length files
    trim_headers(positive_fixed_output, f"{output_prefix}_positive_trimmed.txt")
    trim_headers(negative_fixed_output, f"{output_prefix}_negative_trimmed.txt")

def trim_headers(input_file, output_file):
    """Removes headers from a file, keeping only nucleotide sequences, one per line."""
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            if not line.startswith(">"):
                outfile.write(line.strip() + "\n")


# Example usage
if __name__ == "__main__":
    # BED narrowPeak file paths from 10 different experiments
    #bed_files_paths = ["/users/oishib/ENCFF156CAM.bed", "/users/oishib/ENCFF237YDB.bed", "/users/oishib/ENCFF495BUD.bed", 
    #                   "/users/oishib/ENCFF828FGS.bed", "/users/oishib/ENCFF870FOG.bed", "/users/oishib/ENCFF025ORJ.bed", 
    #                   "/users/oishib/ENCFF188HUC.bed", "/users/oishib/ENCFF402BMP.bed", "/users/oishib/ENCFF561HFN.bed", 
    #                   "/users/oishib/ENCFF863XOL.bed"]
    bed_files_paths = ["/users/oishib/ENCFF401IRH.bed","/users/oishib/ENCFF168UIO.bed","/users/oishib/ENCFF221TMD.bed","/users/oishib/ENCFF544OHJ.bed","/users/oishib/ENCFF781JWJ.bed","/users/oishib/ENCFF889SAK.bed"]
    # Genome reference fasta file (needs to be downloaded separately, e.g., GRCh38.fa)
    genome_fasta = "/users/oishib/ref/hg38.fa"
    
    # Output file prefix
    output_prefix = "dnase_seq_output"
    
    # Fixed length for the sequences
    fixed_length = 120
    
    # Run the process
    process_dnase_seq_data(bed_files_paths, genome_fasta, output_prefix, fixed_length)
